# 系统化的数据问题诊断

在进行数据清洗之前，我们首先需要对数据质量进行全面的诊断。这就像医生在制定治疗方案前，需要先进行全面的体检一样。一个系统化的数据审核流程不仅能帮助我们发现数据中的问题，还能帮助我们理解这些问题的性质、严重程度和潜在影响。

例如，在处理一个电商平台的交易数据时，单纯发现某些订单金额为负数并不足够。我们需要理解：这是数据录入错误？还是退款记录？又或是系统计算误差？只有通过系统化的审核，我们才能做出准确的判断，并选择合适的处理方法。

## 一、数据完整性审核：基础但关键的第一步

### 完整性审核的重要性

数据完整性审核是整个数据质量评估的基础。想象你在分析用户行为数据，如果关键的用户ID或行为时间戳缺失，那么后续的分析就会变得毫无意义。更糟糕的是，如果我们没有及时发现并处理这些缺失值，可能会导致分析结果产生严重偏差。

### 完整性审核的层次

完整性审核需要从多个层次展开：

1. **系统层面的缺失**
   - NULL值：数据库中的真实空值
   - NaN值：Python/Pandas中的特殊空值
   - 空字符串：看似有值但实际为空的情况

2. **业务层面的缺失**
   - "N/A"、"未知"等表示缺失的文本
   - 0或-1等特殊值表示的缺失
   - 明显不合理的默认值（如1900-01-01这样的日期）

3. **上下文层面的缺失**
   - 必填字段的缺失（如订单必须有订单号）
   - 条件性必填字段的缺失（如选择"其他"时必须填写备注）
   - 关联数据的缺失（如有订单号但无对应的订单详情）



## Pandas审核核心函数详解

### 1. isnull() 与缺失值检测

`isnull()`是Pandas中最基础的缺失值检测函数。它的工作原理是遍历DataFrame中的每个元素，检查是否为缺失值（NULL或NaN）。

In [3]:
import pandas as pd
import numpy as np

# 创建示例数据
data = {
    '列1': [1, None, np.nan, 4],
    '列2': ['a', '', None, 'd']
}
df = pd.DataFrame(data)

# 基础检查
print("=== 基础缺失值检查 ===")
print(df.isnull())  # 返回布尔矩阵

# 按列统计缺失值
print("\n=== 每列缺失值统计 ===")
print(df.isnull().sum())

# 缺失值比例
print("\n=== 缺失值比例 ===")
print((df.isnull().sum() / len(df) * 100).round(2), '%')

=== 基础缺失值检查 ===
      列1     列2
0  False  False
1   True  False
2   True   True
3  False  False

=== 每列缺失值统计 ===
列1    2
列2    1
dtype: int64

=== 缺失值比例 ===
列1    50.0
列2    25.0
dtype: float64 %


### 2. sum() 与数据汇总

`sum()`函数不仅可以计算数值的总和，还可以配合布尔值进行计数。

In [4]:
# 数值汇总
print("=== 数值汇总 ===")
numeric_data = pd.DataFrame({
    '销售额': [100, 200, 300, None, 500],
    '数量': [1, 2, None, 4, 5]
})

print("总销售额：", numeric_data['销售额'].sum())
print("总数量：", numeric_data['数量'].sum())

# 条件计数
print("\n=== 条件计数 ===")
# 计算大于200的销售额数量
print("高额销售数：", (numeric_data['销售额'] > 200).sum())

=== 数值汇总 ===
总销售额： 1100.0
总数量： 12.0

=== 条件计数 ===
高额销售数： 2


### 3. select_dtypes() 与数据类型筛选

这个函数允许我们根据数据类型选择特定的列，这在处理大型数据集时特别有用。

In [5]:
# 创建混合类型数据
mixed_data = pd.DataFrame({
    '整数': [1, 2, 3],
    '浮点': [1.1, 2.2, 3.3],
    '文本': ['a', 'b', 'c'],
    '日期': pd.date_range('2024-01-01', periods=3)
})

# 选择数值类型列
print("=== 数值类型列 ===")
numeric_cols = mixed_data.select_dtypes(include=['int64', 'float64'])
print(numeric_cols)

# 选择非数值类型列
print("\n=== 非数值类型列 ===")
non_numeric_cols = mixed_data.select_dtypes(exclude=['int64', 'float64'])
print(non_numeric_cols)

=== 数值类型列 ===
   整数   浮点
0   1  1.1
1   2  2.2
2   3  3.3

=== 非数值类型列 ===
  文本         日期
0  a 2024-01-01
1  b 2024-01-02
2  c 2024-01-03


### 4. str.strip() 与字符串处理

处理文本数据中的空格和特殊字符是数据清洗的重要部分。

In [6]:
# 文本清理示例
text_data = pd.Series([' abc ', 'def  ', '  ghi', ' jkl '])

print("=== 字符串清理 ===")
print("原始数据：")
print(text_data)
print("\n清理后：")
print(text_data.str.strip())

# 检查空字符串
print("\n=== 空字符串检查 ===")
has_empty = text_data.str.strip().eq('')
print("空字符串数量：", has_empty.sum())

=== 字符串清理 ===
原始数据：
0     abc 
1    def  
2      ghi
3     jkl 
dtype: object

清理后：
0    abc
1    def
2    ghi
3    jkl
dtype: object

=== 空字符串检查 ===
空字符串数量： 0


### 5. astype() 与类型转换

数据类型转换是数据预处理的关键步骤。

In [7]:
# 类型转换示例
data = pd.DataFrame({
    '数值字符串': ['1', '2', '3'],
    '分类数据': ['A', 'B', 'A']
})

print("=== 类型转换 ===")
print("原始类型：")
print(data.dtypes)

# 转换为数值
data['数值字符串'] = data['数值字符串'].astype(int)
# 转换为分类
data['分类数据'] = data['分类数据'].astype('category')

print("\n转换后类型")

=== 类型转换 ===
原始类型：
数值字符串    object
分类数据     object
dtype: object

转换后类型
